By Prince Okoli — [GitHub](https://github.com/princyok/deep_learning_without_ml_libraries) — [Blog series](https://princyok.github.io/demonstration-of-the-models-in-action.html)

# Demonstration using the Breast Cancer Wisconsin data

The Breast cancer Wisconsin dataset is a dataset that describe the characteristics of the cell nuclei for 569 breast lumps, and includes whether they are malignant or benign. Published by Clinical Science Center, University of Wisconsin.

Source: https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29

**This Jupyter notebook will put to the test the artificial neuron that was written from scratch without dependence on any ML libraries.** Machine learning libraries may be used in this notebook to make data preprocessing cleaner and more efficient, but not for the actual learning.

In [ ]:
import artificial_neuron as an
import numpy as np
import pandas as pd

from sklearn import model_selection, preprocessing

In [ ]:
data=pd.read_csv("../datasets/breast-cancer-wisconsin-data/data.csv", delimiter=",")


In [ ]:
display(data.head())
display(data.describe())

Convert the target ("daignosis" column) from nominal strings ("M" and "B") to numbers (0 and 1).

In [ ]:
data["diagnosis"]=preprocessing.LabelEncoder().fit(["M","B"]).transform(data["diagnosis"]) # M is 0, B is 1.

data.drop(columns="id",inplace=True)

Convert the pandas Dataframe to numpy array.

In [ ]:
data=data.to_numpy()

Split into train and test sets, and also seperate into features (x) and target (y). Also ensure the shape of the input and output tensors are as expected. For features: num_features x batch_size. For target: 1 x batch_size.

In [ ]:
data_train, data_test = model_selection.train_test_split(data, train_size=0.8, random_state=5)

In [ ]:
print(data_train.shape)
print(data_test.shape)

In [ ]:
data_train=data_train.T
data_test=data_test.T

print(data_train.shape)
print(data_test.shape)

In [ ]:
x_train=data_train[1:, :]
y_train=data_train[0, :].reshape(1,-1)

x_test=data_test[1:, :]
y_test=data_test[0, :].reshape(1,-1)

# Training and evaluation

Just start with some randomly chosen hyperparameters:

* number of iterations: 10

* learning rate (step size): 0.00007

* batch size: 256

In [ ]:
neuron=an.Neuron(X=x_train, Y=y_train)
neuron.train(num_iterations=10, learning_rate=7e-5, batch_size=256)

print("training accuracy: ", neuron.evaluate(X=x_train, Y=y_train, metric="accuracy"))
print("test accuracy:     ", neuron.evaluate(X=x_test, Y=y_test, metric="accuracy"), end="\n\n")

Let's check the activations

In [ ]:
print(neuron.a)

We likely ran into an overflow or underflow somewhere which led to the NaNs. We can try to avoid this by tweaking our hyperparameters.

In [ ]:
vaious_iterations=[50, 100, 500, 1000]
for n in vaious_iterations:
    neuron=an.Neuron(X=x_train, Y=y_train)
    neuron.train(num_iterations=n, learning_rate=7e-6, batch_size=32)

    print("training accuracy: ", neuron.evaluate(X=x_train, Y=y_train, metric="accuracy"))
    print("test accuracy:     ", neuron.evaluate(X=x_test, Y=y_test, metric="accuracy"), end="\n\n")

We can start seeing the training accuracy outpace the test accuracy by the time we are reaching 1000 iterations, so we stop there as that is possibly a sign that we are begining to overfit (a little overfitting is perfectly fine).

**We have 91% test accuracy after 500 iterations.**

Let's play around with the other hyperparameters to see if we can get something better.

In [ ]:
learning_rates=[7e-7,7e-6,7e-5,7e-4]
for lr in learning_rates:    
    neuron=an.Neuron(X=x_train, Y=y_train)
    neuron.train(num_iterations=700, learning_rate=lr, batch_size=32)

    print("training accuracy: ", neuron.evaluate(X=x_train, Y=y_train, metric="accuracy"))
    print("test accuracy:     ", neuron.evaluate(X=x_test, Y=y_test, metric="accuracy"), end="\n\n")

We overflowed/underflowed when our learning rate got relatively too high.

Let's try out different batch sizes. 

In [ ]:
# truncate the printout for np array.
np.set_printoptions(threshold=6)

batch_sizes=[8, 32, 64, 256]
for b in batch_sizes:
    neuron=an.Neuron(X=x_train, Y=y_train)
    neuron.train(num_iterations=500, learning_rate=7e-6, batch_size=b)
    
    print("training accuracy: ", neuron.evaluate(X=x_train, Y=y_train, metric="accuracy"))
    print("test accuracy:     ", neuron.evaluate(X=x_test, Y=y_test, metric="accuracy"))
    print(neuron.a, end="\n\n")

# resetting to the default.
np.set_printoptions(threshold=None)

At this learning are, we are getting good performance with less batch size, but this doesn't mean we will always do better with smaller batch sizes.

Let's shoot for the highest test accuracy possible.

In [ ]:
neuron=an.Neuron(X=x_train, Y=y_train)
neuron.train(num_iterations=1300, learning_rate=9e-6, batch_size=256)

print("training accuracy: ", neuron.evaluate(X=x_train, Y=y_train, metric="accuracy"))
print("test accuracy:     ", neuron.evaluate(X=x_test, Y=y_test, metric="accuracy"), end="\n\n")

**We achieved 92%**. Maybe we can get one or two more % if we keep trying.